In [4]:
import numpy as np
from tqdm.notebook import tqdm
!nvidia-smi

Tue Jan 28 20:37:01 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  On   | 00000000:1A:00.0 Off |                  N/A |
| 30%   32C    P2    58W / 250W |  10671MiB / 11019MiB |      7%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  On   | 00000000:1B:00.0 Off |                  N/A |
| 31%   40C    P2    76W / 250W |  10671MiB / 11019MiB |      6%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [2]:
k = 7

des = str(np.load('prepData/insert2Des__HPGPNRMPC_hg38_chr2.npy'))
anc = str(np.load('prepData/insert2Anc__HPGPNRMPC_hg38_chr2.npy'))

print(len(anc), len(des))

def buildTable(anc, des):
    table = {}
    freq = {}
    for i in tqdm_notebook(range(len(anc)-k*2-1)):
        try:
            table[anc[i:i+k*2+1], des[i+k]] += 1
        except KeyError:
            table[anc[i:i+k*2+1], des[i+k]] = 1
        try:
            freq[anc[i:i+k*2+1]] += 1
        except KeyError:
            freq[anc[i:i+k*2+1]] = 1
    for key in tqdm_notebook(table.keys()):
        table[key] = table[key]/freq[key[0]]
    return table, freq

table, freq = buildTable(anc,des)
    
print(table)
np.save('table_{}'.format(k*2+1), table)

25220017 25220017


/home/mcb/users/dlim63/miniconda3/envs/research/lib/python3.7/site-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


/home/mcb/users/dlim63/miniconda3/envs/research/lib/python3.7/site-packages/ipykernel_launcher.py:20: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [5]:
ancName = '_HPGPNRMPC'
def load_seq(chromList):
    inputAll = ''
    predAll = ''
    outputAll = ''
    for chromosome in chromList:
        try:
            inputAll += str(np.load('prepData/insert2Anc_{}_hg38_chr{}.npy'.format(ancName,chromosome)))#[:10000000]
            outputAll += str(np.load('prepData/insert2Des_{}_hg38_chr{}.npy'.format(ancName,chromosome)))#[:10000000]
            predAll += str(np.load('prepData/simulated_{}_-1_chr{}.npy'.format(ancName, chromosome)))#[:10000000]
        except FileNotFoundError:
            print(chromosome)
            continue
        print(len(inputAll), len(outputAll), len(predAll))
        print(inputAll[-10:], outputAll[-10:], predAll[-10:])
    return [inputAll], [outputAll], [predAll]


inputAll, outputAll, predAll = load_seq([12,16,17,19,20,21,22])


22587896 22587896 22587896
GGCACTGTCT AGCACTGTCT GACACTCTCT
42343685 42343685 42343685
GTGGTGGTGT GTGGTGGTAT GTGGTGGTGA
61140191 61140191 61140191
CAAGAACACA CAAGAACACA CAAGAACACA
79887837 79887837 79887837
AAGCATTTGA AAGCATTTGA AAGCATTTGT
104272401 104272401 104272401
AAGATTTTGG AAGATATTGG AAGATTTTGA
129025047 129025047 129025047
TGTGGAAACA TGTGGAAACA TGTGGAAACT
149860432 149860432 149860432
TAGAGGCCAT TGGAGGCCAT TAGAGGCCAC


In [21]:
np.set_printoptions(precision=15)
def model_simulate(alphabet, k, inputAll):
    inputAll = inputAll[0]
    table15 = np.load('table_15.npy',allow_pickle=True).item()
    table11 = np.load('table_11.npy',allow_pickle=True).item()
    table5 = np.load('table_5.npy',allow_pickle=True).item()
    table1 = np.load('table_1.npy',allow_pickle=True).item()
    
    pred_table = ''
    for i in tqdm_notebook(range(len(inputAll))):
        if i <k:
            pred_table += inputAll[i]
        elif k<=i<len(inputAll)-k:
            mut_prob = []
            for item in alphabet:
                try:
                    mut_prob.append(table15[inputAll[i-7:i+7+1], item])
                except KeyError:
                    try:
                        mut_prob.append(table11[inputAll[i-5:i+5+1], item])
                    except KeyError:
                        try:
                            mut_prob.append(table5[inputAll[i-2:i+2+1], item])
                        except KeyError:
                            try:
                                mut_prob.append(table1[inputAll[i], item])
                            except KeyError:
                                mut_prob.append(0)
#             print(mut_prob)
#             print(np.random.choice(len(mut_prob), 1, p=mut_prob))
#             print(sum(mut_prob))
#             mut_prob = [float(i)/sum(mut_prob) for i in mut_prob]
            mut_prob = np.array(mut_prob)
            mut_prob /= mut_prob.sum()
            
            pred_table += alphabet[np.random.multinomial(1, mut_prob).argmax()]
#             pred_table += alphabet[np.random.choice(len(mut_prob), 1, p=mut_prob)[0]] 
        else :
            pred_table += inputAll[i]
    return pred_table

In [7]:
# k = 7
mut_dict = np.load('mut_dict_insert2.npy',allow_pickle=True).item()
inv_dict = {v: k for k, v in mut_dict.items()}
print(list(inv_dict.keys()))
alphabet = list(inv_dict.keys())

# pred_table = model_simulate(alphabet, k, inputAll)
# np.save('predTable_{}'.format(k*2+1), pred_table)
# print(pred_table[:1000])


['A', 'C', 'G', 'T', '-', '0', '^', 'f', '4', 'B', 'w', 't', 'i', 'e', '.', 'c', '>', 'X', '5', '|', '[', 's', 'u', '(', '=', 'q', 'W', '_', '!', '{', '1', 'Q', 'U', 'b', 'M', ';', 'm', '~', 'a', ']', '&', ')', '7', '6', 'p', '%', 'n', 'P', 'F', 'h', '3', ':', 'z', '8', 'R', 'S', '/', 'j', 'o', 'd', '@', 'L', '}', 'H', 'Y', 'x', '"', 'I', '\\', 'J', '9', '`', 'l', '<', 'N', 'V', 'D', '+', 'E', 'Z', '$', '?', '2', 'K', '*', 'g']


In [16]:
import math
def cross_entropy(alphabet, k, inputAll, outputAll):
    inputAll = inputAll[0]
    outputAll = outputAll[0]
    table = np.load('table_{}.npy'.format(k*2+1),allow_pickle=True).item()
    result = 0
    count = 0
    for i in tqdm_notebook(range(len(inputAll))):
        if k<=i<len(inputAll)-k:
            try:
                result += -math.log(table[inputAll[i-k:i+k+1], outputAll[i]])
                count +=1
            except KeyError:
                continue
#                 print('keyError')
    return result/count

def cross_entropy39(alphabet,inputAll, outputAll):
    inputAll = inputAll[0]
    outputAll = outputAll[0]
    result = 0
    count = 0
    for i in tqdm(range(len(inputAll))):
        if 19<=i<len(inputAll)-19:
            try:
                result += -math.log(table39[inputAll[i-19:i+19+1], outputAll[i]])
                count +=1
            except KeyError:
                try:
                    result += -math.log(table21[inputAll[i-10:i+10+1], outputAll[i]])
                    count +=1
                except KeyError:
                    try:
                        result += -math.log(table15[inputAll[i-7:i+7+1], outputAll[i]])
                        count +=1
                    except KeyError:
                        try:
                            result += -math.log(table11[inputAll[i-5:i+5+1], outputAll[i]])
                            count +=1
                        except KeyError:
                            try:
                                result += -math.log(table5[inputAll[i-2:i+2+1], outputAll[i]])
                                count +=1
                            except KeyError:
                                try:
                                    result += -math.log(table1[inputAll[i], outputAll[i]])
                                    count +=1
                                except KeyError:
                                    result += -math.log(0.01020408163265306)
                                    count +1
    print(result/count)
#                 print('keyError')
    return result/count

def cross_entropy21(alphabet,inputAll, outputAll):
    inputAll = inputAll[0]
    outputAll = outputAll[0]
    result = 0
    count = 0
    for i in tqdm(range(len(inputAll))):
        if 15<=i<len(inputAll)-10:
            try:
                result += -math.log(table21[inputAll[i-10:i+10+1], outputAll[i]])
                count +=1
            except KeyError:
                try:
                    result += -math.log(table15[inputAll[i-7:i+7+1], outputAll[i]])
                    count +=1
                except KeyError:
                    try:
                        result += -math.log(table11[inputAll[i-5:i+5+1], outputAll[i]])
                        count +=1
                    except KeyError:
                        try:
                            result += -math.log(table5[inputAll[i-2:i+2+1], outputAll[i]])
                            count +=1
                        except KeyError:
                            try:
                                result += -math.log(table1[inputAll[i], outputAll[i]])
                                count +=1
                            except KeyError:
                                result += -math.log(0.01020408163265306)
                                count +1
    print(result/count)
#                 print('keyError')
    return result/count

def cross_entropy15(alphabet,inputAll, outputAll):
    inputAll = inputAll[0]
    outputAll = outputAll[0]
    result = 0
    count = 0
    for i in tqdm(range(len(inputAll))):
        if 15<=i<len(inputAll)-7:
            try:
                result += -math.log(table15[inputAll[i-7:i+7+1], outputAll[i]])
                count +=1
            except KeyError:
                try:
                    result += -math.log(table11[inputAll[i-5:i+5+1], outputAll[i]])
                    count +=1
                except KeyError:
                    try:
                        result += -math.log(table5[inputAll[i-2:i+2+1], outputAll[i]])
                        count +=1
                    except KeyError:
                        try:
                            result += -math.log(table1[inputAll[i], outputAll[i]])
                            count +=1
                        except KeyError:
                            result += -math.log(0.01020408163265306)
                            count +1
    print(result/count)
#                 print('keyError')
    return result/count

def cross_entropy11(alphabet,inputAll, outputAll):
    inputAll = inputAll[0]
    outputAll = outputAll[0]
    result = 0
    count = 0
    for i in tqdm(range(len(inputAll))):
        if 15<=i<len(inputAll)-5:
            try:
                result += -math.log(table11[inputAll[i-5:i+5+1], outputAll[i]])
                count +=1
            except KeyError:
                try:
                    result += -math.log(table5[inputAll[i-2:i+2+1], outputAll[i]])
                    count +=1
                except KeyError:
                    try:
                        result += -math.log(table1[inputAll[i], outputAll[i]])
                        count +=1
                    except KeyError:
                        result += -math.log(0.01020408163265306)
                        count +1
    print(result/count)
#                 print('keyError')
    return result/count

def cross_entropy5(alphabet,inputAll, outputAll):
    inputAll = inputAll[0]
    outputAll = outputAll[0]
    result = 0
    count = 0
    for i in tqdm(range(len(inputAll))):
        if 15<=i<len(inputAll)-2:
            try:
                result += -math.log(table11[inputAll[i-5:i+5+1], outputAll[i]])
                count +=1
            except KeyError:
                try:
                    result += -math.log(table1[inputAll[i], outputAll[i]])
                    count +=1
                except KeyError:
                    result += -math.log(0.01020408163265306)
                    count +1
    print(result/count)
#                 print('keyError')
    return result/count

def cross_entropy1(alphabet,inputAll, outputAll):
    inputAll = inputAll[0]
    outputAll = outputAll[0]
    result = 0
    count = 0
    for i in tqdm(range(len(inputAll))):
        if 15<=i<len(inputAll):
            try:
                result += -math.log(table1[inputAll[i], outputAll[i]])
                count +=1
            except KeyError:
                result += -math.log(0.01020408163265306)
                count +1
    print(result/count)
#                 print('keyError')
    return result/count

In [9]:
table39 = np.load('table_39.npy',allow_pickle=True).item()
table21 = np.load('table_21.npy',allow_pickle=True).item()
table15 = np.load('table_15.npy',allow_pickle=True).item()
table11 = np.load('table_11.npy',allow_pickle=True).item()
table5 = np.load('table_5.npy',allow_pickle=True).item()
table1 = np.load('table_1.npy',allow_pickle=True).item()
print(len(table1.keys()))
print(1/98)

246
0.01020408163265306


In [10]:
cross_entropy39(alphabet,inputAll, outputAll)


0.19454299455827476


0.19454299455827476

In [11]:
cross_entropy21(alphabet,inputAll, outputAll)


0.19491412390645477


0.19491412390645477

In [12]:
cross_entropy15(alphabet, inputAll, outputAll)


0.19570464211222716


0.19570464211222716

In [13]:
cross_entropy11(alphabet,inputAll, outputAll)


0.1988374347093904


0.1988374347093904

In [14]:
cross_entropy5(alphabet, inputAll, outputAll)


0.2009239130577984


0.2009239130577984

In [17]:
cross_entropy1(alphabet, inputAll, outputAll)


0.22001797896458855


0.22001797896458855

In [ ]:
0.22000798713506123 0.20882172435559532 0.07722072727808955 0.098972098412289 0.11081867999770367 0.06958409766111845

In [ ]:
cross_entropy2(alphabet, 0, inputAll, outputAll)
cross_entropy2(alphabet, 2, inputAll, outputAll)
cross_entropy2(alphabet, 5, inputAll, outputAll)
cross_entropy2(alphabet, 7, inputAll, outputAll)
cross_entropy2(alphabet, 10, inputAll, outputAll)
cross_entropy2(alphabet, 19, inputAll, outputAll)